In [ ]:
!pip install transformers torch datasets bert-score comet_ml evaluate sacrebleu rouge_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
%cd /content/drive/MyDrive/TST-CycleGAN-main/

## Train style classifier

In [ ]:
import subprocess

import os
os.environ["COMET_API_KEY"] = "???"

import wandb
wandb.login(key="???")


# Define the command and its arguments
command = [
    "python",
    "utils/train_classifier.py",

    "--dataset_path=./data/XFORMAL/family_relationships/",
    "--model_tag=dbmdz/bert-base-italian-cased",
    "--max_sequence_length=64",
    "--batch_size=16",
    "--learning_rate=5e-5",
    "--epochs=5",
    "--lr_scheduler_type=linear",
    "--save_base_folder=./checkpoints/",  # Base folder for saving checkpoints
    "--save_steps=100",
    "--eval_strategy=epoch",
    "--eval_steps=50",
    "--use_cuda_if_available",

    "--comet_logging",
    "--comet_key=???",
    "--comet_workspace=???",
    "--comet_project_name=???"
]

# Run the script using subprocess
result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# Print the output and errors
print("Output:")
print(result.stdout)
print("Errors:")
print(result.stderr)

## Train model

In [ ]:
import subprocess
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


command = [
    "python",
    "train.py",
    "--style_a=informal",
    "--style_b=formal",
    "--lang=it",

    "--path_mono_A=./data/XFORMAL/family_relationships/train.0.txt",
    "--path_mono_B=./data/XFORMAL/family_relationships/train.1.txt",
    "--path_paral_A_eval=./data/XFORMAL/family_relationships/dev.0.txt",
    "--path_paral_B_eval=./data/XFORMAL/family_relationships/dev.1.txt",
    "--path_paral_eval_ref=./data/XFORMAL/family_relationships/references/dev/",
    "--n_references=4",
    "--shuffle",
    "--generator_model_tag=morenolq/bart-it",
    "--discriminator_model_tag=dbmdz/bert-base-italian-cased",
    "--pretrained_classifier_model=./classifiers/XFORMAL/",
    "--lambdas=10|1|1|1|1",
    "--epochs=5",
    "--learning_rate=5e-5",
    "--max_sequence_length=64",
    "--batch_size=32",
    "--save_base_folder=./ckpts/",
    "--save_steps=1",
    "--eval_strategy=epochs",
    "--eval_steps=1",
    "--pin_memory",
    "--use_cuda_if_available",

    "--comet_logging",
    "--comet_key=???",
    "--comet_workspace=???",
    "--comet_project_name=???"
]

result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
print(result.stdout)
print(result.stderr)

## Test model

In [ ]:
import subprocess

command = [
    "python",
    "test.py",
    "--style_a=informal",
    "--style_b=formal",
    "--lang=it",

    "--path_paral_A_test=./data/XFORMAL/family_relationships/test.0.txt",
    "--path_paral_B_test=./data/XFORMAL/family_relationships/test.1.txt",
    "--path_paral_test_ref=./data/XFORMAL/family_relationships/references/test/",
    "--n_references=4",
    "--generator_model_tag=morenolq/bart-it",
    "--discriminator_model_tag=dbmdz/bert-base-italian-cased",
    "--pretrained_classifier_eval=./classifiers/XFORMAL/",
    "--from_pretrained=./ckpts/",
    "--num_workers=2",
    "--max_sequence_length=64",
    "--batch_size=16",
    "--pin_memory",
    "--use_cuda_if_available",

    "--comet_logging",
    "--comet_key=???",
    "--comet_workspace=???",
    "--comet_project_name=???"
]

result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
print(result.stdout)
print(result.stderr)